In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import  statsmodels as stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
df=pd.read_csv('/kaggle/input/ushealthinsurancedataset/insurance.csv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#Top Rows of Dataframe

df.head()

In [3]:
#Assigning Gender to unique vectors for comparison

male = df.query('sex == "male"')['charges']
female = df.query('sex == "female"')['charges']

from statsmodels.stats.weightstats import stats

stats.shapiro(male)
stats.shapiro(female)

stats.levene(male, female)


In [8]:
#Two Sample T Test (difference in means)

from statsmodels.stats.weightstats import ttest_ind

ttest_ind(male, female)

In [8]:
#Overlapped Histogram of Charges By Gender

df.groupby('sex').charges.hist(legend=True, stacked=True)

In [9]:
#Basic Correlation Matrix

df.corr()

In [3]:
#ADD DUMMY VARIABLES FOR CATEGORICAL FEATURES
#Remove Colinear / secondary variables

df_new= pd.get_dummies(df, columns=["smoker", "sex", "region"]) #Smokin status, gender, and region get dummy variables 
df_final=df_new.drop(['sex_male', 'smoker_no'], axis = 1)

df_final.head()

In [34]:
## HeatMap for Correlation Matrix between all variables in dataframe

import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df_final.corr(), annot = True);


In [45]:
#Top Rows of Updated Dataframe
df_final.head()

In [4]:

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

#Split into explanatory and response variables
X = df_final[['age','sex_female', 'smoker_yes', 'bmi', 'children','region_northwest','region_northeast','region_southwest']]
y = df_final['charges']

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42) 

lm_model = LinearRegression(normalize=True) # Instantiate
lm_model.fit(X_train, y_train) #Fit
        
#Predict and score the model
y_test_preds = lm_model.predict(X_test) 

#Rsquared and y_test
rsquared_score = r2_score(y_test, y_test_preds)
length_y_test = len(y_test)

"The r-squared score for your model was {} on {} values.".format(rsquared_score, length_y_test)

In [7]:
#Output coefficients from linear model

lm_model.coef_
lm_model.intercept_